<a href="https://colab.research.google.com/github/ahamednazeer/AI/blob/master/heart_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Step 1: Install required packages
!pip install streamlit pyngrok pandas numpy scikit-learn matplotlib seaborn

# Step 2: Install ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin/

# Step 3: Clear old ngrok configuration
!rm -f /root/.config/ngrok/ngrok.yml

# Step 4: Set up ngrok authtoken with the new token
!ngrok authtoken 2YsmHMOy8PrZC6e17Q0XxAVdvTt_6iSPDiqyJfh2Uu4hZtvqb

# Step 5: Write the Streamlit app code to a file
app_code = """
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns

# Set page configuration
st.set_page_config(page_title="Heart Disease Predictor", layout="wide")

# Function to load and preprocess data
@st.cache_data
def load_and_preprocess_data():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
    columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
               'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
    df = pd.read_csv(url, names=columns)

    df.replace('?', np.nan, inplace=True)
    df.dropna(inplace=True)

    df['ca'] = df['ca'].astype(float)
    df['thal'] = df['thal'].astype(float)

    X = df.drop('target', axis=1)
    y = df['target'].apply(lambda x: 1 if x > 0 else 0)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler, X.columns

# Function to train model
@st.cache_resource
def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

# Function to get model metrics
def get_model_metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    return accuracy, precision, recall

# Main Streamlit app
def main():
    st.title("Heart Disease Prediction System")

    X_train_scaled, X_test_scaled, y_train, y_test, scaler, feature_names = load_and_preprocess_data()
    model = train_model(X_train_scaled, y_train)

    st.sidebar.header("Navigation")
    page = st.sidebar.radio("Select Page", ["Prediction", "Model Metrics"])

    if page == "Prediction":
        st.header("Patient Data Input")

        col1, col2 = st.columns(2)

        with col1:
            age = st.number_input("Age", min_value=1, max_value=120, value=50)
            sex = st.selectbox("Sex", ["Male", "Female"])
            cp = st.selectbox("Chest Pain Type", ["Typical Angina", "Atypical Angina",
                                                "Non-anginal Pain", "Asymptomatic"])
            trestbps = st.number_input("Resting Blood Pressure (mm Hg)", min_value=50, max_value=200, value=120)
            chol = st.number_input("Cholesterol (mg/dl)", min_value=100, max_value=600, value=200)
            fbs = st.selectbox("Fasting Blood Sugar > 120 mg/dl", ["No", "Yes"])
            restecg = st.selectbox("Resting ECG Results", ["Normal", "ST-T Wave Abnormality",
                                                         "Left Ventricular Hypertrophy"])

        with col2:
            thalach = st.number_input("Maximum Heart Rate", min_value=60, max_value=220, value=150)
            exang = st.selectbox("Exercise Induced Angina", ["No", "Yes"])
            oldpeak = st.number_input("ST Depression", min_value=0.0, max_value=10.0, value=1.0, step=0.1)
            slope = st.selectbox("Slope of Peak Exercise ST Segment", ["Upsloping", "Flat", "Downsloping"])
            ca = st.number_input("Number of Major Vessels (0-3)", min_value=0, max_value=3, value=0)
            thal = st.selectbox("Thalassemia", ["Normal", "Fixed Defect", "Reversible Defect"])

        sex_val = 1 if sex == "Male" else 0
        cp_val = {"Typical Angina": 1, "Atypical Angina": 2, "Non-anginal Pain": 3, "Asymptomatic": 4}[cp]
        fbs_val = 1 if fbs == "Yes" else 0
        restecg_val = {"Normal": 0, "ST-T Wave Abnormality": 1, "Left Ventricular Hypertrophy": 2}[restecg]
        exang_val = 1 if exang == "Yes" else 0
        slope_val = {"Upsloping": 1, "Flat": 2, "Downsloping": 3}[slope]
        thal_val = {"Normal": 3, "Fixed Defect": 6, "Reversible Defect": 7}[thal]

        input_data = np.array([[age, sex_val, cp_val, trestbps, chol, fbs_val, restecg_val,
                               thalach, exang_val, oldpeak, slope_val, ca, thal_val]])

        input_scaled = scaler.transform(input_data)

        if st.button("Predict"):
            prediction = model.predict(input_scaled)[0]
            probability = model.predict_proba(input_scaled)[0][1]

            st.subheader("Prediction Results")
            risk_level = "High" if probability > 0.7 else "Medium" if probability > 0.3 else "Low"
            st.write(f"**Risk Level:** {risk_level}")
            st.write(f"**Probability of Heart Disease:** {probability:.2%}")
            st.write(f"**Prediction:** {'Positive' if prediction == 1 else 'Negative'} for heart disease")

    else:
        st.header("Model Performance Metrics")
        accuracy, precision, recall = get_model_metrics(model, X_test_scaled, y_test)

        col1, col2, col3 = st.columns(3)
        col1.metric("Accuracy", f"{accuracy:.2%}")
        col2.metric("Precision", f"{precision:.2%}")
        col3.metric("Recall", f"{recall:.2%}")

        st.subheader("Feature Importance")
        feature_importance = pd.DataFrame({
            'feature': feature_names,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)

        fig, ax = plt.subplots()
        sns.barplot(x='importance', y='feature', data=feature_importance)
        plt.title("Feature Importance in Heart Disease Prediction")
        st.pyplot(fig)

if __name__ == "__main__":
    main()
"""

# Write the app code to app.py
with open("app.py", "w") as f:
    f.write(app_code)

# Step 6: Run Streamlit and ngrok
from pyngrok import ngrok
import subprocess

# Start Streamlit app in the background
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

# Create ngrok tunnel
public_url = ngrok.connect(8501, bind_tls=True)
print("Public URL:", public_url)

--2025-07-18 15:01:19--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 13.248.244.96, 35.71.179.82, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9260570 (8.8M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.3’

ngrok-v3-stable-lin 100%[===================>]   8.83M  9.48MB/s    in 0.9s    

2025-07-18 15:01:20 (9.48 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.3’ saved [9260570/9260570]

ngrok
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://fee42b454c49.ngrok-free.app" -> "http://localhost:8501"
